In [205]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [206]:
data = pd.read_csv("post_EDA.csv")
print("after" + str(data.shape))
data['severity'] = data['severity'] - 1
#data = data[data.severity != 0]

indicies_2 = data[data.severity == 2].index
indicies_1 = data[data.severity == 1].index
indicies_0 = data[data.severity == 0].index

print(indicies_2)
print(indicies_1)
print(indicies_0)

sample_size = sum(data.severity == 2)
random_indices_0 = np.random.choice(indicies_0, sample_size, replace=False)
random_indices_1 = np.random.choice(indicies_1, sample_size, replace=False)
random_indices_2 = np.random.choice(indicies_2, sample_size, replace=False)


data1 = data.loc[random_indices_0]
data2 = data.loc[random_indices_1]
data3 = data.loc[random_indices_2]

#data = pd.concat([data1, data2, data3])
print(data)

print("after" + str(data.shape))

after(230188, 16)
Int64Index([   261,    754,   1157,   1158,   1331,   1927,   2199,   2362,
              2593,   3066,
            ...
            230139, 230140, 230141, 230142, 230143, 230144, 230145, 230184,
            230185, 230186],
           dtype='int64', length=4899)
Int64Index([     5,     54,     57,     68,     89,    172,    173,    204,
               205,    232,
            ...
            230163, 230168, 230174, 230176, 230177, 230178, 230179, 230180,
            230181, 230187],
           dtype='int64', length=78117)
Int64Index([     0,      1,      2,      3,      4,      6,      7,      8,
                 9,     10,
            ...
            230166, 230167, 230169, 230170, 230171, 230172, 230173, 230175,
            230182, 230183],
           dtype='int64', length=147172)
        severity  speed_limit  intersection  weather  lighting  month  \
0              0           60          True        0         0      1   
1              0           60          Tr

In [207]:
null = data.isnull().sum()
null_sorted = null.sort_values(ascending = False)
print(null_sorted)
data = data.dropna()

severity          0
speed_limit       0
intersection      0
weather           0
lighting          0
month             0
day_of_week       0
hour              0
pedestrian        0
vehicle_other     0
car               0
large_vehicles    0
rail_vehicle      0
two_wheeled       0
time_of_day       0
season            0
dtype: int64


In [208]:
X  = data.drop(columns = "severity")
Y = data['severity']

In [209]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 123, test_size = .3)

In [210]:
log_model = LogisticRegression()

In [211]:
scaler = StandardScaler()

In [212]:
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [213]:
X_test_scaled = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

In [214]:
X_train_final, y_train_final = SMOTE().fit_resample(X_train_scaled, y_train)
np.bincount(y_train_final)

array([102910, 102910, 102910], dtype=int64)

# **Log Reg**

In [215]:
steps =  [('pca', PCA(n_components=3)), ('m', LogisticRegression())]
model = Pipeline(steps = steps)

In [216]:
model.fit(X_train_scaled, y_train)

Pipeline(steps=[('pca', PCA(n_components=3)), ('m', LogisticRegression())])

In [217]:
y_pred_log = model.predict(X_test_scaled)

In [218]:
print(classification_report(y_test, y_pred_log))

              precision    recall  f1-score   support

           0       0.67      0.93      0.78     44262
           1       0.51      0.15      0.24     23302
           2       0.00      0.00      0.00      1493

    accuracy                           0.65     69057
   macro avg       0.39      0.36      0.34     69057
weighted avg       0.60      0.65      0.58     69057



C:\Users\thr0w\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thr0w\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thr0w\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [219]:
def create_model(neurons= 5, optimizer = 'Adam', activation = 'sigmoid', layer_count = 2, learning_rate = .003):
  model = Sequential()    
  model.add(Dense(neurons, activation = activation, input_shape = (15, ), kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4), activity_regularizer=regularizers.L2(1e-5)))
  for i in range(layer_count - 1):    
    model.add(Dense(neurons, activation=activation, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4), activity_regularizer=regularizers.L2(1e-5)))
  model.add(Dense(1, activation='hard_sigmoid'))  
  model.compile(loss='categorical_crossentropy', optimizer= optimizer, metrics=['categorical_accuracy'])
  K.set_value(model.optimizer.learning_rate, learning_rate)
  return model

In [220]:
#nn = create_model()
#nn.fit(X_train_scaled, y_train, epochs = 200, verbose = 0)

In [221]:
#y_pred_net = nn.predict(X_test_scaled)

In [222]:
#y_pred_net = np.argmax(y_pred_net, axis=-1)

In [223]:
#print(classification_report(y_test, y_pred_net))

In [224]:
from sklearn.utils import resample

params = {
    'criterion' : ['ginni', 'entropy'],
    'n_estimators' : [10, 50, 100, 200],
    'max_depth' : [4, 5, 6, 10, 15, 20],
    'max_features' : [None, 'sqrt', 'log2'],
    'min_impurity_decrease' : [0, 0.0005, 0.0007, 0.001, 0.01],
    'bootstrap' : [False, True]
}
search = GridSearchCV(RandomForestClassifier(), params, n_jobs=-2, cv=5)
X_train_sampled, y_train_sampled = resample(X_train_scaled, y_train)
model = search.fit(X_train_sampled, y_train_sampled)
best = model.best_estimator_.get_params()

C:\Users\thr0w\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
3600 fits failed out of a total of 7200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1800 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\thr0w\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\thr0w\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\ensemble\_forest.py", line 476, in fit
    trees = Parallel(
  File "C:\Users\thr0w\anaconda3\envs\tf-gpu\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):

In [225]:
rf = RandomForestClassifier(n_estimators = best['n_estimators'], 
                               max_depth = best['max_depth'],
                               max_features = best['max_features'], 
                               min_impurity_decrease = best['min_impurity_decrease'],
                               bootstrap = best['bootstrap'],
                               criterion = best['criterion']
                              )
rf.fit(X_train_sampled, y_train_sampled)

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=20,
                       min_impurity_decrease=0)

In [226]:
y_pred_rf = rf.predict(X_test_scaled)

In [227]:
#y_pred_rf = np.argmax(y_pred_rf, axis=-1)

In [228]:
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.73      0.87      0.80     44262
           1       0.62      0.41      0.49     23302
           2       0.51      0.19      0.28      1493

    accuracy                           0.70     69057
   macro avg       0.62      0.49      0.52     69057
weighted avg       0.69      0.70      0.68     69057



In [229]:
dt = DecisionTreeClassifier()

In [230]:
dt.fit(X_train_scaled, y_train)

DecisionTreeClassifier()

In [231]:
y_pred_dt = dt.predict(X_test_scaled)

In [232]:
print(classification_report(y_test, y_pred_dt))

              precision    recall  f1-score   support

           0       0.76      0.84      0.80     44262
           1       0.62      0.50      0.55     23302
           2       0.39      0.31      0.35      1493

    accuracy                           0.71     69057
   macro avg       0.59      0.55      0.56     69057
weighted avg       0.70      0.71      0.70     69057



In [244]:
#ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
params = {
    "base_estimator" : [None, DecisionTreeClassifier()],
    "n_estimators" : [1, 25, 50, 100, 200, 500],
    "learning_rate" : [1.0, 0.1, 0.01, .001],
    "algorithm" : ["SAMME", "SAMME.R"]
}
search = GridSearchCV(AdaBoostClassifier(), params, n_jobs=-2, cv = 2)
X_train_sampled, y_train_sampled = resample(X_train_scaled, y_train)
X_train_sampled, X_valid, y_train_sampled, y_valid = train_test_split(X_train_sampled, y_train_sampled, train_size=0.9)
model = search.fit(X_valid, y_valid)


best = model.best_estimator_.get_params()
ada = AdaBoostClassifier(base_estimator = best["base_estimator"],
                        n_estimators = best["n_estimators"],
                        learning_rate = best["learning_rate"],
                        algorithm = best["algorithm"])

In [245]:
ada.fit(X_train_sampled, y_train_sampled)

AdaBoostClassifier(n_estimators=500)

In [246]:
y_pred_ada = ada.predict(X_test_scaled)

In [247]:
print(classification_report(y_test, y_pred_ada))

              precision    recall  f1-score   support

           0       0.69      0.89      0.78     44262
           1       0.54      0.28      0.36     23302
           2       0.53      0.01      0.01      1493

    accuracy                           0.66     69057
   macro avg       0.58      0.39      0.38     69057
weighted avg       0.63      0.66      0.62     69057



In [248]:
from sklearn.neighbors import KNeighborsClassifier

In [249]:
params = {
    "n_neighbors" : [2, 5, 7, 10],
    "metric" : ["manhattan", "minkowski"],
    "weights" : ["distance", "uniform"],
    "leaf_size" : [10, 15, 20, 25, 30, 50]
}
search = GridSearchCV(KNeighborsClassifier(), params, n_jobs=-2, cv = 3)
X_train_sampled, y_train_sampled = resample(X_train_scaled, y_train)
model = search.fit(X_train_sampled, y_train_sampled)
best = model.best_estimator_.get_params()

knn = KNeighborsClassifier(n_neighbors = best["n_neighbors"],
                           metric = best["metric"],
                           weights = best["weights"],
                           leaf_size = best["leaf_size"])
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=10, weights='distance')

In [250]:
y_pred_knn = knn.predict(X_test_scaled)

In [251]:
print(classification_report(y_test, y_pred_knn))

              precision    recall  f1-score   support

           0       0.76      0.84      0.80     44262
           1       0.62      0.52      0.56     23302
           2       0.54      0.29      0.38      1493

    accuracy                           0.72     69057
   macro avg       0.64      0.55      0.58     69057
weighted avg       0.71      0.72      0.71     69057

